In [1]:
import os
import logging
logging.disable(logging.WARNING) 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import json
import pandas as pd
%cd ..
from src.hierarchy import *
%cd -
import tensorflow as tf
import IPython

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


In [2]:
def clip_df(df_path, max_images):
    df = pd.read_json(df_path)
    df = df[:max_images]
    df.to_json(df_path)

def mock_environment(training_dir, s3_path, max_images=40):
    if not os.path.exists(training_dir):
        !mkdir {training_dir}
        !aws s3 cp {s3_path} {training_dir} --recursive > /dev/null
    clip_df(training_dir + "/dataframes/training.json", max_images)
    clip_df(training_dir + "/dataframes/testing.json", int(max_images * 0.1))
    
    return ["--train", training_dir, '--brightness_range', str(0.03),  "--batch_size", str(32), '--channel_shift_range', str(0.07), '--combination_type', str(1), "--dropout", str(0.3), 
            '--horizontal_flip', str(True), "--learning_rate", str(1e-3), '--max_count', str(10), '--num_epochs', str(1), '--num_finetune_epochs', str(1), 
            '--num_lock_layers', str(100), '--rotation_range', str(15), '--shear_range', str(0.06), '--validation_split', str(0.1), 
            "--sm-model-dir", "/home/ec2-user/SageMaker/data-science-product-image/files/model-files", "--weight_decay", str(0.04), '--zoom_range', str(0.06)]


def run_mock_training(commands):
    commands_str = " ".join(commands)
    !python ../src/concatenated_model_trainer.py {commands_str}

In [3]:
training_dir = "/home/ec2-user/SageMaker/data-science-product-image/notebooks/dummy-training"
s3_path = 's3://data-science-training-datasets/product-image/mturk-50k'

commands = mock_environment(training_dir, s3_path, 100)

In [4]:
run_mock_training(commands)

Using TensorFlow backend.
100%|███████████████████████████████████████████| 31/31 [00:00<00:00, 54.30it/s]
Found 320 validated image filenames belonging to 2 classes.
Found 10 validated image filenames belonging to 2 classes.
Found 10 validated image filenames belonging to 2 classes.
Train for 10 steps, validate for 1 steps
10/10 [==============================] - 50s 5s/step - loss: 46.9623 - accuracy: 0.6781 - hierarchy: 0.6319 - fallback: 0.6812 - val_loss: 79.5568 - val_accuracy: 0.1000 - val_hierarchy: 0.1500 - val_fallback: 0.1000
probabilities shape: (10, 73)
one hot shape: (10, 73)
Train for 10 steps, validate for 1 steps
10/10 [==============================] - 46s 5s/step - loss: 26.4897 - accuracy: 0.8750 - hierarchy: 0.8924 - fallback: 0.8781 - val_loss: nan - val_accuracy: 0.0000e+00 - val_hierarchy: 0.2498 - val_fallback: 0.0000e+00
Validation performance:
1/1 [==============================] - 1s 728ms/step - loss: inf - accuracy: 0.1000 - hierarchy: 0.1500 - fallback: 0